# Full Example
THe idea with this NB is to do the same as the pymc_prophet_example NB but do everything (holidays, regressors etc...) in a single example and compare pymc_prohet with prophet. Then we will use this to build the Bayesian PyMC component.

In [1]:
import pandas as pd
from pymcprophet.model import BayesTS, BayesTSConfig
from prophet import Prophet


Importing plotly failed. Interactive plots will not work.


In [2]:
df = df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df['ds'] = pd.to_datetime(df['ds'])

## Holidays
These are user defined. We will also add auto US ones.

In [3]:
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

## Regressor
Custom dataset for Sundays during the NFL Season.

In [4]:
def nfl_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and (date.month > 8 or date.month < 2):
        return 1
    else:
        return 0

df['nfl_sunday'] = df['ds'].apply(nfl_sunday)

## Modelling

In [5]:
train_df = df.query("ds.dt.year <= 2014")
test_df = df.query("ds.dt.year > 2014")

### PyMC

In [6]:
ts_config = BayesTSConfig()
ts_model = BayesTS(ts_config)
ts_model.add_holiday_country("US")
ts_model.add_holiday(holidays)
ts_model.add_regressor("nfl_sunday")

ts_model.assign_model_matrix(train_df)

ts_model.model_df.columns

Index(['ds', 'y', 'nfl_sunday', 'y_trans', 't', 't_seasonality',
       'yearly_sin_1', 'yearly_sin_2', 'yearly_sin_3', 'yearly_sin_4',
       'yearly_sin_5', 'yearly_sin_6', 'yearly_sin_7', 'yearly_sin_8',
       'yearly_sin_9', 'yearly_sin_10', 'yearly_cos_1', 'yearly_cos_2',
       'yearly_cos_3', 'yearly_cos_4', 'yearly_cos_5', 'yearly_cos_6',
       'yearly_cos_7', 'yearly_cos_8', 'yearly_cos_9', 'yearly_cos_10',
       'weekly_sin_1', 'weekly_sin_2', 'weekly_sin_3', 'weekly_cos_1',
       'weekly_cos_2', 'weekly_cos_3', 'playoff', 'playoff_lag_1', 'superbowl',
       'superbowl_lag_1', 'christmas_day', 'christmas_day_observed',
       'columbus_day', 'independence_day', 'independence_day_observed',
       'juneteenth_national_independence_day',
       'juneteenth_national_independence_day_observed', 'labor_day',
       'martin_luther_king_jr_day', 'memorial_day', 'new_years_day',
       'new_years_day_observed', 'thanksgiving', 'veterans_day',
       'veterans_day_observed', 'was

### Prophet

In [7]:
prophet_model = Prophet(holidays=holidays)
prophet_model.add_country_holidays(country_name='US')
prophet_model.add_regressor('nfl_sunday')

In [8]:
transformed = prophet_model.preprocess(train_df)

In [9]:
transformed.X.columns

Index(['yearly_delim_1', 'yearly_delim_2', 'yearly_delim_3', 'yearly_delim_4',
       'yearly_delim_5', 'yearly_delim_6', 'yearly_delim_7', 'yearly_delim_8',
       'yearly_delim_9', 'yearly_delim_10', 'yearly_delim_11',
       'yearly_delim_12', 'yearly_delim_13', 'yearly_delim_14',
       'yearly_delim_15', 'yearly_delim_16', 'yearly_delim_17',
       'yearly_delim_18', 'yearly_delim_19', 'yearly_delim_20',
       'weekly_delim_1', 'weekly_delim_2', 'weekly_delim_3', 'weekly_delim_4',
       'weekly_delim_5', 'weekly_delim_6', 'Christmas Day (observed)_delim_+0',
       'Christmas Day_delim_+0', 'Columbus Day_delim_+0',
       'Independence Day (observed)_delim_+0', 'Independence Day_delim_+0',
       'Labor Day_delim_+0', 'Martin Luther King Jr. Day_delim_+0',
       'Memorial Day_delim_+0', 'New Year's Day (observed)_delim_+0',
       'New Year's Day_delim_+0', 'Thanksgiving_delim_+0',
       'Veterans Day (observed)_delim_+0', 'Veterans Day_delim_+0',
       'Washington's Birthday